In [ ]:
import pandas as pd
from src.constants import (
    PREPROCESSING_PATH,
    TARGET
)

import numpy as np
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import power_transform
from utils.classes.extractor import FunctionalGroupToGramExtractor

from typing import (
    Optional,
    List,
    Any
)
from sklearn.model_selection import KFold, RepeatedKFold, RepeatedStratifiedKFold

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout
)
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import ModelCheckpoint


Train = pd.read_csv(f"{PREPROCESSING_PATH}/train_merged_CIF/Train.csv")
Train = Train.set_index(Train.columns[0])
Train = Train.replace([np.inf], 99999999)
Train = Train.replace([-np.inf], -99999999)
Train = Train.dropna()


Train = Train.drop(Train[Train["surface_area [m^2/g]"]<=0].index)
Train['density'] = (Train["weight [u]"] / Train["volume [A^3]"]) * 1.66054
ratio = 0.7

train = Train.iloc[: int(ratio * Train.shape[0])]

test = Train.iloc[int(ratio * Train.shape[0]):]



X = train[
    [
        "heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]",
        "CO2/N2_selectivity",
        "_cell_volume",
        "_cell_length_a",
        "_cell_length_b",
        "_cell_length_c",
        "density"
    ]
]

y = train[['void_fraction']]

X_test = test[
    [
        "heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]",
        "CO2/N2_selectivity",
        "_cell_volume",
        "_cell_length_a",
        "_cell_length_b",
        "_cell_length_c",
        "density"
    ]
]

y_test = test[['void_fraction']]
# y_test = test["surface_area [m^2/g]"]  # **0.5]

2021-09-24 16:49:06.237121: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-24 16:49:06.237160: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# model_checkpoint_callback = ModelCheckpoint(
#     filepath="best_model.h5",
#     monitor='val_loss',
#     mode='min',
#     save_best_only=True)


# def wider_model():
# 	# create model
# 	model = Sequential()
# 	# model.add(Dense(120, activation='sigmoid'))
# 	model.add(Dense(60, activation='sigmoid'))
# 	model.add(Dense(60, activation='sigmoid'))
# 	model.add(Dense(60, activation='sigmoid'))
# 	model.add(Dense(60, activation='sigmoid'))
# 	model.add(Dense(30, activation='sigmoid'))
# 	model.add(Dense(1, activation='sigmoid'))
# 	# Compile model
# 	model.compile(loss='mae', optimizer='nadam')
# 	return model


# model = wider_model()
# hist = model.fit(
#     X,
#     y,
#     epochs=100,
#     validation_data=(X_test, y_test),
# 	callbacks=[model_checkpoint_callback]
# )

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('best_model.h5')

prediction = pd.DataFrame(model.predict(X_test))
prediction.columns = y_test.columns
prediction.index = y_test.index
prediction

2021-09-24 16:49:11.809028: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-24 16:49:11.809241: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-24 16:49:11.809254: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-24 16:49:11.809270: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-cf8541de-dbc3-45f6-bc1e-4fa446cacbcd): /proc/driver/nvidia/version does not exist
2021-09-24 16:49:11.809465: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operatio

,void_fraction
Unnamed: 0,
47630,0.388589
47631,0.273713
47632,0.347425
47633,0.194464
47635,0.391269
...,...
66520,0.196374
66521,0.520175
66522,0.518002


In [ ]:
mean_absolute_error(
    (prediction["void_fraction"]) ,
    (y_test["void_fraction"])
)

0.06739468527089239

In [ ]:
max(np.abs(prediction["void_fraction"] - y_test["void_fraction"]))

0.5810849476051332

In [ ]:
y_test[["void_fraction"]]

,void_fraction
Unnamed: 0,
47630,0.36297
47631,0.22833
47632,0.35320
47633,0.14975
47635,0.36629
...,...
66520,0.30190
66521,0.66340
66522,0.57051


In [ ]:
prediction[["void_fraction"]]

,void_fraction
Unnamed: 0,
47630,0.388589
47631,0.273713
47632,0.347425
47633,0.194464
47635,0.391269
...,...
66520,0.196374
66521,0.520175
66522,0.518002


In [ ]:
from xgboost import XGBRegressor

reg = XGBRegressor()
reg.fit(X.values, y.values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
mean_absolute_error(reg.predict(X_test), y_test)

0.04765743596161957

In [ ]:
pd.DataFrame(reg.predict(X_test))

,0
0,0.361899
1,0.234947
2,0.325208
3,0.163438
4,0.387328
...,...
16129,0.273043
16130,0.566652
16131,0.474311
16132,0.561576


In [ ]:
X_final = Train[
    [
        "heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]",
        "CO2/N2_selectivity",
        "_cell_volume",
        "_cell_length_a",
        "_cell_length_b",
        "_cell_length_c",
        "density"
    ]
]
y_final = Train[['void_fraction']]

xgboost = XGBRegressor()
xgboost.fit(X_final.values, y_final.values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
mean_absolute_error(xgboost.predict(X_final), y_final)

0.02096953274114096

In [ ]:
xgboost.save_model("imputer/xgboost/void_fraction.json")

In [ ]:
model.save("imputer/keras/void_fractio.h5")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf8541de-dbc3-45f6-bc1e-4fa446cacbcd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>